# Notebook 1: NPU Fundamentals and Benchmarking

## The Hybrid NPU Stack Series

This notebook accompanies **Part 1 & 2** of the blog series on Hybrid NPU Stacks. We'll explore:

1. Understanding processor differences (CPU vs GPU vs NPU)
2. Benchmarking inference performance
3. Measuring power efficiency
4. Detecting available hardware accelerators

---

## 1. Setup and Dependencies

First, let's install the necessary packages. We'll use lightweight libraries that work across different hardware configurations.

In [ ]:
# Install required packages
# Uncomment and run if needed
# !pip install numpy pandas matplotlib psutil onnxruntime

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import psutil
import platform
from typing import Dict, List
from dataclasses import dataclass
import warnings
warnings.filterwarnings('ignore')

print(f"Python version: {platform.python_version()}")
print(f"NumPy version: {np.__version__}")
print(f"Platform: {platform.platform()}")

## 2. Understanding Processor Architectures

### Key Differences: CPU vs GPU vs NPU vs TPU

Let's create a visual comparison of different processor types and their characteristics.

In [ ]:
@dataclass
class ProcessorProfile:
    """Profile of a processor type for AI workloads"""
    name: str
    best_for: str
    power_watts_min: float
    power_watts_max: float
    tops_min: float
    tops_max: float
    typical_use_case: str
    parallelism: str
    precision: str

# Define processor profiles based on 2025 industry data
processors = [
    ProcessorProfile(
        name="CPU",
        best_for="Sequential tasks, control logic",
        power_watts_min=15,
        power_watts_max=150,
        tops_min=0.1,
        tops_max=1.0,
        typical_use_case="General compute, orchestration",
        parallelism="Low (4-64 cores)",
        precision="FP32, FP64"
    ),
    ProcessorProfile(
        name="GPU",
        best_for="Parallel workloads, training",
        power_watts_min=75,
        power_watts_max=700,
        tops_min=100,
        tops_max=3000,
        typical_use_case="Training, complex inference",
        parallelism="Very High (thousands of cores)",
        precision="FP32, FP16, INT8"
    ),
    ProcessorProfile(
        name="NPU",
        best_for="Inference, always-on AI",
        power_watts_min=1,
        power_watts_max=15,
        tops_min=10,
        tops_max=80,
        typical_use_case="Edge inference, mobile AI",
        parallelism="Medium (optimized matrix ops)",
        precision="INT8, INT4, FP16"
    ),
    ProcessorProfile(
        name="TPU",
        best_for="Large-scale ML",
        power_watts_min=200,
        power_watts_max=450,
        tops_min=200,
        tops_max=400,
        typical_use_case="Cloud training/inference",
        parallelism="High (systolic arrays)",
        precision="BF16, INT8"
    )
]

# Create comparison DataFrame
df_processors = pd.DataFrame([
    {
        'Processor': p.name,
        'Best For': p.best_for,
        'Power (W)': f"{p.power_watts_min}-{p.power_watts_max}",
        'TOPS Range': f"{p.tops_min}-{p.tops_max}",
        'Use Case': p.typical_use_case,
        'Parallelism': p.parallelism,
        'Precision': p.precision
    }
    for p in processors
])

print("\n" + "="*80)
print("PROCESSOR COMPARISON MATRIX")
print("="*80)
display(df_processors)

In [ ]:
# Visualize TOPS per Watt efficiency
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot 1: Power consumption range
colors = ['#3182ce', '#e53e3e', '#38a169', '#805ad5']
names = [p.name for p in processors]
power_min = [p.power_watts_min for p in processors]
power_max = [p.power_watts_max for p in processors]

x = np.arange(len(names))
axes[0].bar(x, power_max, color=colors, alpha=0.3, label='Max')
axes[0].bar(x, power_min, color=colors, label='Min')
axes[0].set_xlabel('Processor Type')
axes[0].set_ylabel('Power Consumption (Watts)')
axes[0].set_title('Power Consumption Range')
axes[0].set_xticks(x)
axes[0].set_xticklabels(names)
axes[0].set_yscale('log')

# Plot 2: TOPS range
tops_min = [p.tops_min for p in processors]
tops_max = [p.tops_max for p in processors]

axes[1].bar(x, tops_max, color=colors, alpha=0.3, label='Max')
axes[1].bar(x, tops_min, color=colors, label='Min')
axes[1].set_xlabel('Processor Type')
axes[1].set_ylabel('TOPS (Trillion Operations/Second)')
axes[1].set_title('Compute Performance Range')
axes[1].set_xticks(x)
axes[1].set_xticklabels(names)
axes[1].set_yscale('log')

# Plot 3: TOPS per Watt (efficiency)
efficiency = []
for p in processors:
    avg_tops = (p.tops_min + p.tops_max) / 2
    avg_power = (p.power_watts_min + p.power_watts_max) / 2
    efficiency.append(avg_tops / avg_power)

axes[2].bar(x, efficiency, color=colors)
axes[2].set_xlabel('Processor Type')
axes[2].set_ylabel('TOPS per Watt')
axes[2].set_title('Energy Efficiency (Higher is Better)')
axes[2].set_xticks(x)
axes[2].set_xticklabels(names)

plt.tight_layout()
plt.savefig('processor_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n📊 KEY INSIGHT: NPUs offer the best TOPS-per-Watt efficiency,")
print("   making them ideal for battery-powered edge devices.")

## 3. Detecting Hardware Accelerators

Let's build a utility to detect what AI acceleration hardware is available on your system.

In [ ]:
def detect_hardware_accelerators() -> Dict[str, any]:
    """
    Detect available hardware accelerators for AI inference.
    Returns a dictionary with detected hardware and capabilities.
    """
    results = {
        'cpu': {},
        'gpu': {},
        'npu': {},
        'onnx_providers': []
    }
    
    # CPU Detection
    results['cpu'] = {
        'processor': platform.processor(),
        'cores_physical': psutil.cpu_count(logical=False),
        'cores_logical': psutil.cpu_count(logical=True),
        'architecture': platform.machine(),
        'available': True
    }
    
    # GPU Detection (PyTorch)
    try:
        import torch
        if torch.cuda.is_available():
            results['gpu'] = {
                'available': True,
                'type': 'CUDA',
                'device_count': torch.cuda.device_count(),
                'device_name': torch.cuda.get_device_name(0) if torch.cuda.device_count() > 0 else 'N/A'
            }
        elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
            results['gpu'] = {
                'available': True,
                'type': 'Apple MPS (Metal)',
                'device_count': 1,
                'device_name': 'Apple Silicon GPU'
            }
        else:
            results['gpu']['available'] = False
    except ImportError:
        results['gpu']['available'] = False
        results['gpu']['note'] = 'PyTorch not installed'
    
    # ONNX Runtime Provider Detection
    try:
        import onnxruntime as ort
        results['onnx_providers'] = ort.get_available_providers()
        
        # Check for NPU-related providers
        npu_providers = ['DmlExecutionProvider', 'QNNExecutionProvider', 
                        'CoreMLExecutionProvider', 'OpenVINOExecutionProvider']
        detected_npu = [p for p in results['onnx_providers'] if p in npu_providers]
        
        if detected_npu:
            results['npu'] = {
                'available': True,
                'providers': detected_npu
            }
        else:
            results['npu']['available'] = False
            
    except ImportError:
        results['onnx_providers'] = ['Not installed']
        results['npu']['available'] = False
    
    return results

# Run detection
hardware = detect_hardware_accelerators()

print("\n" + "="*60)
print("HARDWARE ACCELERATOR DETECTION RESULTS")
print("="*60)

print("\n🖥️  CPU:")
for key, value in hardware['cpu'].items():
    print(f"   {key}: {value}")

print("\n🎮 GPU:")
for key, value in hardware['gpu'].items():
    print(f"   {key}: {value}")

print("\n🧠 NPU:")
for key, value in hardware['npu'].items():
    print(f"   {key}: {value}")

print("\n📦 ONNX Runtime Providers:")
for provider in hardware['onnx_providers']:
    print(f"   - {provider}")

## 4. Benchmarking Neural Network Operations

Let's benchmark the core operations that NPUs optimize for: matrix multiplication.

In [ ]:
def benchmark_matmul(sizes: List[int], iterations: int = 10) -> pd.DataFrame:
    """
    Benchmark matrix multiplication at different sizes.
    This is the core operation in neural network inference.
    """
    results = []
    
    for size in sizes:
        # Create random matrices (simulating weights and activations)
        A = np.random.randn(size, size).astype(np.float32)
        B = np.random.randn(size, size).astype(np.float32)
        
        # Warmup
        _ = np.matmul(A, B)
        
        # Benchmark
        times = []
        for _ in range(iterations):
            start = time.perf_counter()
            _ = np.matmul(A, B)
            end = time.perf_counter()
            times.append(end - start)
        
        avg_time = np.mean(times) * 1000  # Convert to ms
        std_time = np.std(times) * 1000
        
        # Calculate GFLOPS (2 * N^3 for matrix multiply)
        flops = 2 * (size ** 3)
        gflops = (flops / (avg_time / 1000)) / 1e9  # GFLOPS
        
        results.append({
            'Matrix Size': f"{size}x{size}",
            'Time (ms)': f"{avg_time:.3f} ± {std_time:.3f}",
            'GFLOPS': f"{gflops:.2f}",
            'Operations': f"{flops/1e9:.2f}B"
        })
        
    return pd.DataFrame(results)

# Run benchmarks
print("\n" + "="*60)
print("MATRIX MULTIPLICATION BENCHMARK (CPU/NumPy)")
print("="*60)
print("\nThis simulates the core operation in neural network layers.")
print("NPUs optimize specifically for this type of computation.\n")

sizes = [128, 256, 512, 1024, 2048]
df_benchmark = benchmark_matmul(sizes)
display(df_benchmark)

In [ ]:
def benchmark_precision_comparison(size: int = 1024, iterations: int = 10) -> pd.DataFrame:
    """
    Compare performance across different numeric precisions.
    NPUs excel at lower precision (INT8, FP16) operations.
    """
    precisions = [
        ('float64', np.float64, 'FP64 - Full precision'),
        ('float32', np.float32, 'FP32 - Standard'),
        ('float16', np.float16, 'FP16 - Half precision (NPU optimized)'),
    ]
    
    results = []
    base_time = None
    
    for name, dtype, description in precisions:
        A = np.random.randn(size, size).astype(dtype)
        B = np.random.randn(size, size).astype(dtype)
        
        # Warmup
        _ = np.matmul(A, B)
        
        # Benchmark
        times = []
        for _ in range(iterations):
            start = time.perf_counter()
            _ = np.matmul(A, B)
            end = time.perf_counter()
            times.append(end - start)
        
        avg_time = np.mean(times) * 1000
        memory_mb = (A.nbytes + B.nbytes) / (1024 * 1024)
        
        if base_time is None:
            base_time = avg_time
        
        results.append({
            'Precision': name,
            'Description': description,
            'Time (ms)': f"{avg_time:.3f}",
            'Memory (MB)': f"{memory_mb:.1f}",
            'Relative Speed': f"{base_time / avg_time:.2f}x"
        })
    
    return pd.DataFrame(results)

print("\n" + "="*60)
print("PRECISION COMPARISON BENCHMARK")
print("="*60)
print(f"\nMatrix size: 1024x1024")
print("NPUs are optimized for lower precision (FP16, INT8) operations.\n")

df_precision = benchmark_precision_comparison()
display(df_precision)

print("\n💡 KEY INSIGHT: Lower precision = faster inference + less memory.")
print("   This is why NPU quantization (INT8, INT4) is so powerful.")

## 5. Edge vs Cloud Inference Simulation

Let's model what happens when you split inference between edge NPU and cloud GPU.

In [ ]:
@dataclass
class InferenceConfig:
    """Configuration for inference simulation"""
    name: str
    compute_time_ms: float
    network_latency_ms: float
    cost_per_inference: float
    power_watts: float

# Define inference configurations
edge_npu = InferenceConfig(
    name="Edge NPU",
    compute_time_ms=15,
    network_latency_ms=0,
    cost_per_inference=0.0,
    power_watts=5
)

cloud_gpu = InferenceConfig(
    name="Cloud GPU",
    compute_time_ms=8,
    network_latency_ms=50,
    cost_per_inference=0.00005,
    power_watts=300
)

def simulate_inference_batch(config: InferenceConfig, batch_size: int = 1000) -> Dict:
    """Simulate a batch of inference requests."""
    total_time_ms = batch_size * (config.compute_time_ms + config.network_latency_ms)
    total_cost = batch_size * config.cost_per_inference
    
    return {
        'config': config.name,
        'batch_size': batch_size,
        'total_time_seconds': total_time_ms / 1000,
        'avg_latency_ms': config.compute_time_ms + config.network_latency_ms,
        'total_cost': total_cost,
        'throughput_per_sec': batch_size / (total_time_ms / 1000)
    }

# Run simulations
print("\n" + "="*60)
print("EDGE vs CLOUD INFERENCE SIMULATION")
print("="*60)

batch_sizes = [1000, 10000, 100000, 1000000]

results_edge = [simulate_inference_batch(edge_npu, batch) for batch in batch_sizes]
results_cloud = [simulate_inference_batch(cloud_gpu, batch) for batch in batch_sizes]

# Create comparison table
comparison_data = []
for e, c in zip(results_edge, results_cloud):
    savings = ((c['total_cost'] - e['total_cost']) / max(c['total_cost'], 0.01)) * 100
    comparison_data.append({
        'Batch Size': f"{e['batch_size']:,}",
        'Edge Latency (ms)': e['avg_latency_ms'],
        'Cloud Latency (ms)': c['avg_latency_ms'],
        'Edge Cost': f"${e['total_cost']:.2f}",
        'Cloud Cost': f"${c['total_cost']:.2f}",
        'Cost Savings': f"{savings:.0f}%"
    })

df_comparison = pd.DataFrame(comparison_data)
display(df_comparison)

In [ ]:
# Visualize the cost comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

batch_labels = [f"{b//1000}K" if b < 1000000 else f"{b//1000000}M" for b in batch_sizes]
x = np.arange(len(batch_sizes))
width = 0.35

edge_costs = [r['total_cost'] for r in results_edge]
cloud_costs = [r['total_cost'] for r in results_cloud]

# Cost comparison
axes[0].bar(x - width/2, [max(c, 0.1) for c in edge_costs], width, label='Edge NPU', color='#38a169')
axes[0].bar(x + width/2, cloud_costs, width, label='Cloud GPU', color='#e53e3e')
axes[0].set_xlabel('Batch Size')
axes[0].set_ylabel('Total Cost ($)')
axes[0].set_title('Inference Cost: Edge NPU vs Cloud GPU')
axes[0].set_xticks(x)
axes[0].set_xticklabels(batch_labels)
axes[0].legend()
axes[0].set_yscale('log')

# Latency comparison
axes[1].bar(x - width/2, [15] * len(batch_sizes), width, label='Edge NPU', color='#38a169')
axes[1].bar(x + width/2, [58] * len(batch_sizes), width, label='Cloud GPU', color='#e53e3e')
axes[1].set_xlabel('Batch Size')
axes[1].set_ylabel('Latency per Request (ms)')
axes[1].set_title('Per-Request Latency Comparison')
axes[1].set_xticks(x)
axes[1].set_xticklabels(batch_labels)
axes[1].legend()

plt.tight_layout()
plt.savefig('edge_vs_cloud_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n📊 ANALYSIS:")
print(f"   • Edge NPU: Zero cloud cost, {edge_npu.compute_time_ms}ms latency")
print(f"   • Cloud GPU: $50/million requests, {cloud_gpu.compute_time_ms + cloud_gpu.network_latency_ms}ms latency")

## 6. NPU Hardware Landscape (2025)

In [ ]:
# Current NPU hardware landscape (2025 data)
npu_hardware = [
    {'Vendor': 'Qualcomm', 'Product': 'Snapdragon X2 Elite', 'NPU': 'Hexagon', 'TOPS': 80, 'Target': 'Laptops'},
    {'Vendor': 'Apple', 'Product': 'M4 Chip', 'NPU': 'Neural Engine', 'TOPS': 38, 'Target': 'Mac/iPad'},
    {'Vendor': 'Intel', 'Product': 'Panther Lake', 'NPU': 'Intel NPU 4', 'TOPS': 50, 'Target': 'Laptops'},
    {'Vendor': 'AMD', 'Product': 'Ryzen AI Max', 'NPU': 'XDNA 2', 'TOPS': 50, 'Target': 'Workstations'},
    {'Vendor': 'Google', 'Product': 'Tensor G4', 'NPU': 'Edge TPU', 'TOPS': 15, 'Target': 'Pixel phones'},
    {'Vendor': 'Google', 'Product': 'Coral NPU', 'NPU': 'RISC-V', 'TOPS': 0.5, 'Target': 'IoT/Wearables'}
]

df_hardware = pd.DataFrame(npu_hardware)
print("\n" + "="*70)
print("NPU HARDWARE LANDSCAPE (2025)")
print("="*70)
display(df_hardware)

# Visualization
fig, ax = plt.subplots(figsize=(12, 6))
vendors = [f"{h['Vendor']}\n{h['Product']}" for h in npu_hardware]
tops_values = [h['TOPS'] for h in npu_hardware]
colors = ['#4299e1', '#667eea', '#3182ce', '#e53e3e', '#38a169', '#48bb78']

bars = ax.barh(vendors, tops_values, color=colors)
ax.set_xlabel('TOPS (Trillion Operations Per Second)')
ax.set_title('NPU Performance Comparison (2025)')

for bar, val in zip(bars, tops_values):
    ax.text(val + 1, bar.get_y() + bar.get_height()/2, f'{val} TOPS', va='center')

plt.tight_layout()
plt.savefig('npu_tops_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Key Takeaways

1. **NPUs are purpose-built for inference** - They optimize matrix operations at low precision
2. **Edge inference eliminates network latency** - 0ms vs 50-200ms for cloud
3. **Cost scales dramatically** - 1M requests: Cloud ~$50, Edge ~$0
4. **Hardware is maturing rapidly** - 50-80 TOPS NPUs now standard
5. **Lower precision = higher efficiency** - FP16/INT8 faster than FP32

In [ ]:
print("\n" + "="*60)
print("NOTEBOOK 1 COMPLETE")
print("="*60)
print("\n✅ Processor comparison analyzed")
print("✅ Hardware detection utility created")
print("✅ Matrix multiplication benchmarked")
print("✅ Edge vs Cloud cost simulation complete")
print("✅ NPU hardware landscape visualized")
print("\n➡️  Next: Notebook 2 - Task Routing and Model Partitioning")